In [2]:
# Load packages
library(tidyverse)
library(wesanderson)
library(vegan)
library(fs)

# Description of data and sample sets

All analyses in this notebook were performed separately on two different sample sets: 

1. _highErrorRemoved_: This sample set contains 515 samples from urban and rural habitats across 26 cities (25 cities for GLUE-LOW1 plus 20 samples from Toronto. 5 samples were removed due to having abnormaly high alignment error rates (as reported by `Qualimap`). These samples likely represent different species that were mistaken for white clover
2. _finalSamples_lowCovRemoved_: This sample set contains 432 samples across 26 cities. In addition to the 5 samples with high error rates above, an additional 83 samples with low coverage (< 0.31X) were removed since these were previously found to have low mapping % and lower than average SNP density when compared to remaining samples. 

In addition to the 500 samples sequenced as part of GLUE-LOW1, we are including 10 urban and 10 rural plants from Toronto. These were sequenced to higher coverage (~12X) as part of a separate project. We selected these plants as follows:

- 2 samples from each of 5 urban populations for a total of 10 urban plants
- 5 samples from one rural population and 1 sample from each of 5 other populations. These samples come from the western Toronto transect only. We couldn't evenly sample rural populations because they weren't evenly sampled for the Toronto data. Total of 10 rural plants 
- All 20 plants were downsampled by sampling 25% of their reads. Because initial coverage between these samples varied between ~7.5X and ~14X, downsampled coverages in GLUE should range from ~1.8X to 3.5X.

# Global patterns of diversity and structure

In this notebook, we'll analyze the output fron `ANGSD` run separately on both sample sets described above. The notebook contains the following analyses:

1. Sequencing depth across all samples. This is mostly a check to see whether our maximum depth threshold used in genotype likelihood and SFS estimation is reasonable.
2. General shape of the SFS estimated using all sites, 4-fold sites, and 0-fold sites across the genome
3. Genome-wide average pairwise genetic diversity (Theta<sub>pi</sub>) across all samples. Done for all sites, 4-fold sites, and 0-fold sites.
4. PCA showing population clustering of samples using 4-fold sites only. 

    - The PCA is performed on both sample sets and at 3 different MAF cutoffs to assess the impact of including rare variants on sample clustering.

## Sequencing depth

- Examine total sequencing depth across samples to get a sense of whether our max depth cutoff in `ANGSD` is reasonable. 
- Only looking at depth across chromosome 1 since this should be enough to see what's going on

### Load in depth dataset

In [3]:
load_depth_df <- function(path){
    
    # Get name of folder with parameter combinations
    sample_set <- str_split(path_dir(path), '/', simplify = TRUE)[1,1]

    # Read in SFS
    full_path <- paste0(inpath, '/', path)
    depth_df <- suppressMessages(
        read_delim(full_path, delim = '\t', col_names = FALSE)) %>% 
    t() %>% 
    as.data.frame() %>% 
    rename('num_sites' = 'V1') %>% 
    mutate(cov = 1:n() - 1,
           sample_set = sample_set)
    return(depth_df)
}

In [4]:
inpath <- '../results/angsd/depth/'
depth_df <- list.files(inpath, pattern = 'CM019101.1_\\w+_allSites.depthGlobal', recursive = TRUE) %>% 
  map_dfr(., load_depth_df)

In [5]:
head(depth_df)

The dataset as 20,000 rows. Each row represents the total number of sites with a total X coverage, for one or the other sample set (10,000 rows per sample set). The exception is bin 10,000, which represents the number of sites with coverage equal or greater than 10,000. 

### Histogram of sequencing coverage

Plotting histogram of number of sites at each X coverage along chromosome 1.

Red dashed line is the max depth cutoff used when estimating genotype likelihoods and the SFS. Sites with depth great than this cutoff were excluded. 

- This depth (1250X) was calculated as 2 x the mean coverage from qualimap (1.25X) x Number of samples (515)
- Max depth cutoff is the same for both sample sets since even though there are fewer samples in the *finalSamples* sample set, they are at higher mean coverage, so these cancel out. 

In [6]:
depth_cutoff <- 1250
depthGlobal_plot <- ggplot(depth_df, aes(x = cov, y = num_sites)) +
    geom_bar(stat = 'identity', color = 'black', fill = 'white') + 
    xlab('Coverage') + ylab('Number of sites') +
    facet_wrap(~sample_set) + 
    theme_classic() + 
    geom_vline(xintercept = depth_cutoff, linetype = 'dashed', color = 'red') +
    theme(axis.text = element_text(size = 13),
        axis.title = element_text(size = 15))
depthGlobal_plot

In [7]:
# Estimate proportion of sites above and below cutoff for both sample sets
total_sites <- sum(depth_df$num_sites, na.rm = TRUE)
depth_df %>% 
    mutate(is_below_cutoff = ifelse(cov <= depth_cutoff, 1, 0)) %>% 
    group_by(sample_set, is_below_cutoff) %>% 
    summarize(n = sum(num_sites, na.rm = TRUE)) %>% 
    mutate(freq = n / sum(n, na.rm = TRUE))

In [8]:
# Save plot
outpath <- snakemake@output[[1]]
print(outpath)
ggsave(filename = outpath, plot = depthGlobal_plot, device = 'pdf', 
       dpi = 300, width = 14, height = 8, units = 'in')

### My take

Regardless of sample set:
- Most sites have 1 to 2X coverage
- As coverage increases beyond 2X, there is a gradual decline in the number of sites for a given depth
- 1250X is a reasonable cutoff and captures ~98% of all sites. 

## SFS and diversity

Here I'll plot the SFS for all sites across the genome, in additional to subsets of degenerate sites (4-fold and 0-fold). We'll similarly estimate diversity as the genome-wide average of pairwise nucleotide differences for these same sets of sites. 

- Filters applied on sites:
    - Minimum phred-scaled read mapping quality of 30
    - Minimimum phred-scaled base quality of 20
    - Max depth of 1250X across all individuals
    - 50% of all individuals required to have at least 1 read
    
Genotype likelihoods are estimated using the `samtools` model with the `samtools` "extended BAQ" algorithm to recalibrate base quality scores around INDELS. 

- During tests on the high coverage Toronto data, the `GATK` GL model called more sites than the `samtools` GL model, especially singletons. However, the overall shape of the SFS was consistent under both models. 
- The `extended BAQ` model to recalibrate base quality scores performed well in tests on the Toronto data. The `standard BAQ` model (`-baq 1` in `ANGSD`) was too conservative and removed a lot of sites. Not performing BAQ is not recommended since, unlike haplotype-aware callers (e.g., `GATK` and `freebayes`), there is no local realignment of reads around indels in `ANGSD`.

### SFS

#### Load in SFS data as single dataframe

In [9]:
load_long_sfs <- function(path){
  
    # Get name of folder with parameter combinations
    dir <- str_split(path_dir(path), '/', simplify = TRUE)
    sample_set <- dir[1, 1]
    site <- dir[1, 2]

    # Read in SFS
    full_path <- paste0(inpath, '/', path)
    sfs <- suppressMessages(read_delim(full_path, delim= '\t', col_names = FALSE)) %>% 
    as.data.frame() %>% 
    rename('maf' = 'X1',
           'num_sites' = 'X2') %>% 
    filter(num_sites != 0) %>%  #  folded SFS so samples > # of samples will be 0
    mutate(prop_sites = num_sites / sum(num_sites),
           sample_set = sample_set,
           site = site)
    return(sfs)
}

In [10]:
inpath <- '../results/angsd/sfs/'
sfs_df <- list.files(inpath, pattern = '*allChroms.sfs', recursive = TRUE) %>% 
  map_dfr(., load_long_sfs)

In [11]:
# Quick look at the da
head(sfs_df)

#### Plot SFS

- Only plotting minor allele frequency from 1 to 25. Therefore, invariant sites and sites with MAF > 25/N not shown

In [12]:
# How many sites in each category?
sfs_df %>% 
    group_by(sample_set, site) %>% 
    summarize(total_sites = sum(num_sites))

In [13]:
sfs_plot <- sfs_df %>% 
    filter(maf != 0 & maf <= 25)  %>%
    ggplot(., aes(x = maf, y = prop_sites)) + 
    geom_bar(stat ='identity', color = 'black',  width=.70) + 
    facet_grid(sample_set~ site) +
    ylab('Proportion of sites') + xlab('Minor allele frequency') +
    scale_fill_manual(values = cols) +
    scale_x_continuous(breaks = seq(1, 40, 3)) +
    scale_y_continuous(breaks = seq(0, 0.13, 0.02)) + 
    theme_classic() + 
    theme(axis.text = element_text(size = 13),
        axis.title = element_text(size = 15))
sfs_plot

In [14]:
# Save plot
outpath <- snakemake@output[[2]]
print(outpath)
ggsave(filename = outpath, plot = sfs_plot, device = 'pdf', 
       dpi = 300, width = 14, height = 14, units = 'in')

#### My take

- Regardless of site type, the *finalSamples_lowCovRemoved* SFSs look better than the *highErrorRemoved* sample set
    - Likely due to fewer sites in the *highErrorRemoved* sample set, which reduces `ANGSD`'s "power" to find the ML estimate of the SFS
    - Despite having more samples in the *highErrorRemoved* sample set, there are fewer sites because the proportion of individuals required to have reads at a site was fixed at 50% for both sample sets. For the *finalSamples* dataset, this equates to 216 samples, which is esily met in this dataset since bad individuals have already been removed. As a result, more sites pass this filter. By contrast, ~80 samples in the *highErrorRemoved* sample set are pretty low coverage, so this filter is more strict in that case. 
- The *finalSamples* SFSs look decent and show the expected patterns (e.g., more strongly left-skewed 0fold SFS). 
- There is still some jaggedness, some of which could probably be alleviated by further increasing the number of sites by, for example, further reducing the proportion of individuals required to have reads at a site.
    - However, given the low depth (~1X) and high variation in depth across samples (range: 1X to ~7X), some jaggedness is likely inevitable. 
    - In addition, keep in mind this SFS include samples from all across the world. Some of the jaggedness may come from including global samples with different demographic histories and structures. 

#### Nicer plot of *finalSamples_lowCovRemoved* SFS

In [15]:
cols <- wes_palette("Darjeeling1", n = 3, type = 'discrete')
sfs_plot <- sfs_df %>% 
    filter(maf != 0 & maf <= 25)  %>%
    filter(sample_set == 'finalSamples_lowCovRemoved')  %>%
    ggplot(., aes(x = maf, y = prop_sites, fill = site)) + 
    geom_bar(stat ='identity',  width=.75, position = 'dodge') + 
    ylab('Proportion of sites') + xlab('Minor allele frequency') +
    scale_fill_manual(values = cols) +
    scale_x_continuous(breaks = seq(1, 40, 3)) +
    scale_y_continuous(breaks = seq(0, 0.13, 0.02)) + 
    theme_classic() + 
    theme(axis.text = element_text(size = 13),
        axis.title = element_text(size = 15))
sfs_plot

In [16]:
# Save plot
outpath <- snakemake@output[[3]]
print(outpath)
ggsave(filename = outpath, plot = sfs_plot, device = 'pdf', 
       dpi = 300, width = 8, height = 12, units = 'in')

### Diversity

#### Load in diversity dataframes

In [17]:
load_div_neut_df <- function(path){
  
    # Get name of folder with parameter combinations
    dir <- str_split(path_dir(path), '/', simplify = TRUE)
    sample_set <- dir[1, 1]
    site <- dir[1, 2]

    # Read in stats
    full_path <- paste0(inpath, '/', path)
    stats <- suppressMessages(read_delim(full_path, delim= '\t', col_names = TRUE)) %>% 
    mutate(sample_set = sample_set,
           site = site,
           tp_scaled = tP / nSites,
           tw_scaled = tW / nSites)
    return(stats)
  
}

In [23]:
inpath <- '../results/angsd/summary_stats/thetas/'
thetas_df <- list.files(inpath, pattern = '*diversityNeutrality.thetas.idx.pestPG', recursive = TRUE) %>% 
  map_dfr(., load_div_neut_df)

In [25]:
# tp is theta_pi
# tw is theta_waterson
# td is Tajima's D
theta_summary <- thetas_df %>% 
  group_by(sample_set, site) %>% 
  summarise(total_sites = sum(nSites),
            mean_tp = mean(tp_scaled),
            mean_tw = mean(tw_scaled), 
            mean_td = mean(Tajima))
theta_summary

In [26]:
outpath <- snakemake@output[[4]]
print(outpath)
write_delim(x = theta_summary, file = outpath, delim = '\t', col_names = TRUE)

#### My take

Regardless of sample set:

- Looks good. Lower diversity at 0-fold sites, as expected
- ~1.5% genome-wide diversity

## PCA

- PCA is performed on genotype likelihoods
- Filtering criteria the same as those above with one exception:
    - This is based only on variant sites with 3 different MAF cutoffs: 0.005, 0.01, 0.05
- PCAs were performed using `PCAngsd`, which generates a covariance matrix of allele frequencies across samples
    - We can extract eigenvectors from this covariance matrix and plot in R. 

### Load in data and perform PCA

- Perform PCA on all combinations of sample set and MAF cutoff. 
- Merge PCA results into single dataframe

#### A few functions

In [27]:
# Function to get PCA summary as matrix
pca_importance <- function(covMat) {
    
    # Sample set and MAF info
    sample_set <- covMat %>% distinct(sample_set) %>% pull()
    maf <- covMat %>% distinct(maf) %>% pull()
    
    # Prepare matrix for PCA
    not_all_na <- function(x) any(!is.na(x))
    covMat_matrix <- covMat %>% 
        dplyr::select(where(not_all_na)) %>% 
        dplyr::select(-sample_set, -maf) %>% 
        as.matrix()
    
    pca_summary <- summary(princomp(covMat_matrix))
    vars <- pca_summary$sdev^2
    vars <- vars/sum(vars)
    df_out <- rbind(
        `Proportion of Variance` = vars, 
        `Cumulative Proportion` = cumsum(vars)) %>%
    as.data.frame() %>% 
    dplyr::select(Comp.1:Comp.4) %>% 
    mutate(sample_set = sample_set,
           maf = maf)
    
    return(df_out)
}

In [28]:
# Function to load allele frequency covariance matrix and return as dataframe for tidy joining
load_covMat <- function(path){
    
    # Get sample set and MAF info for allele frequency covariance matrix from PCAngsd
    basename <- basename(path)
    sample_set <- str_split(basename, '_4fold', simplify = TRUE)[1,1]
    maf <- str_extract(basename, '(?<=maf).+(?=_pcangsd)')
    
    # Load in covariance matrix, assign sample set and maf as columns
    full_path <- paste0(inpath, '/', path)
    covMat_out <- suppressMessages(
        read_delim(
            full_path, 
            col_names = FALSE, 
            delim = ' ')) %>% 
    as.data.frame() %>% 
    mutate(sample_set = sample_set,
           maf = maf)
    
    return(covMat_out)
    
}

In [29]:
# Function to create sample sheet for specific sample set
get_sample_sheet <- function(sample_set){
    
    # Load habitat info
    # Hard-coded since not expected to change
    habitat_info <- suppressMessages(
        read_delim(
            '../../sequencing-prep/resources/low1_sampleSheet.txt', 
            delim = '\t')) %>% 
        dplyr::select(continent, range, city, pop, individual, site, sample)
    
    # Hard coded relative path since not expected to change
    sample_sheet_path <- '../results/program_resources/'
    
    # Full path to sample sheet by sample set
    full_path <- paste0(sample_sheet_path, 'angsd_', sample_set, '_order.txt')
    
    # Load in sample sheet
    sample_sheet_out <- suppressMessages(
        read_table(
            full_path, 
            col_names = FALSE) %>% 
        rename('sample' = 'X1')) %>%
        left_join(., habitat_info, by = 'sample')
    
    return(sample_sheet_out)
    
}

In [30]:
# Performs PCA on covariance matrix and return dataframe with samples scores along first 4 PCs
perform_pca <- function(covMat){
    
    # Sample set and MAF info
    sample_set <- covMat %>% distinct(sample_set) %>% pull()
    maf <- covMat %>% distinct(maf) %>% pull()
    
    # Prepare matrix for PCA
    not_all_na <- function(x) any(!is.na(x))
    covMat_matrix <- covMat %>% 
        dplyr::select(where(not_all_na)) %>% 
        dplyr::select(-sample_set, -maf) %>% 
        as.matrix()
    
    samples <- get_sample_sheet(sample_set)
    
    # Perform PCA and return samples scores along first 4 PCs
    eigenvectors <- eigen(covMat_matrix)
    eigen_df <- eigenvectors$vectors %>% 
        as.data.frame() %>% 
        dplyr::select(V1, V2, V3, V4) %>% 
        rename('PC1' = 'V1',
               'PC2' = 'V2', 
               'PC3' = 'V3',
               'PC4' = 'V4') %>% 
    bind_cols(., samples) %>% 
    mutate(sample_set = sample_set,
           maf = maf)
    
    return(eigen_df)
    
}

#### Perform and plot the PCAs

Here is a breakdown on the number of sites. Reminder: these are 4fold degenerate sites.

- finalSamples_lowCovRemoved & MAF = 0.005: 487,260
- finalSamples_lowCovRemoved & MAF = 0.01: 415,672
- finalSamples_lowCovRemoved & MAF = 0.05: 230,212
- highErrorRemoved & MAF = 0.005: 284,545
- highErrorRemoved & MAF = 0.01: 243,647
- highErrorRemoved & MAF = 0.05: 135,774

In [31]:
# Load covariance matrices as single df
inpath <- '../results/population_structure/pcangsd'
covMat_df <- list.files(inpath, pattern = '*.cov', recursive = TRUE) %>% 
    map_dfr(., load_covMat)

In [32]:
# Scores of samples along first 4 PCs for all samples sets and MAF cutoffs as single df
eigen_df <- covMat_df %>% 
    group_by(sample_set, maf) %>% 
    group_split(.) %>% 
    map_dfr(perform_pca)

In [33]:
# Percent variance explained by first 4 PCs for all sample_set by MAF combinations
covMat_df %>% 
    group_by(sample_set, maf) %>% 
    group_split(.) %>% 
    map_dfr(pca_importance)

#### Colored by Habitat

In [35]:
col1 <- wes_palette("Darjeeling1", n = 5, type = 'discrete')[2]
col2 <- wes_palette("Darjeeling1", n = 5, type = 'discrete')[4]
cols <- c(col1, col2)
pca_plot <- eigen_df %>% 
    ggplot(., aes(x = PC1, y = PC2, color = site, shape = site)) +
        geom_point(size = 2) + 
        scale_color_manual(values = cols) + 
        facet_grid(sample_set~maf) +
        theme_classic() + 
        xlab('PC1') + ylab('PC2') +
        scale_x_continuous(breaks = seq(-0.10, 0.10, 0.10)) +
        theme(axis.text = element_text(size = 13),
             axis.title = element_text(size = 15),
             legend.position = 'top')
pca_plot

In [36]:
# Save plot
outpath <- snakemake@output[[5]]
print(outpath)
ggsave(filename = outpath, plot = pca_plot, device = 'pdf', 
       dpi = 300, width = 14, height = 14, units = 'in')

#### Quick take

- Comparing rows within columns (i.e., sample sets for same MAF), there is little difference between the sample sets in terms of the relative positions of samples and percent variation explained by each PC
- In contrast to the SFS estimation, this suggests that we can use the full sample set (i.e., *highErrorRemoved*) for the PCA. Probably because PCA doesn't actually need that many sites. 

**For the remaining plots, I'll focus only on the *highErrorRemoved* sample set**

#### Colored by city

In [37]:
# Plot
cols <- wes_palette("Darjeeling1", n = 26, type = 'continuous')
pca_byCity <- eigen_df %>% 
    filter(sample_set == 'highErrorRemoved') %>% 
    ggplot(., aes(x = PC1, y = PC2, color = city, shape = site)) +
        geom_point(size = 2) + 
        scale_color_manual(values = cols) + 
        scale_x_continuous(breaks = seq(-0.10, 0.10, 0.10)) +
        facet_wrap(~maf) +
        theme_classic() + 
        xlab('PC1') + ylab('PC2') +
        theme(axis.text = element_text(size = 13),
            axis.title = element_text(size = 15),
            legend.position = 'top')
pca_byCity

In [38]:
# Save plot
outpath <- snakemake@output[[6]]
print(outpath)
ggsave(filename = outpath, plot = pca_byCity, device = 'pdf', 
       dpi = 300, width = 14, height = 14, units = 'in')

#### Colored by continent

In [40]:
# Plot
cols <- wes_palette("Darjeeling1", n = 6, type = 'continuous')
pca_byContinent <- eigen_df %>% 
    filter(sample_set == 'highErrorRemoved') %>% 
    ggplot(., aes(x = PC1, y = PC2, color = continent, shape = site)) +
        geom_point(size = 2) + 
        scale_color_manual(values = cols) + 
        scale_x_continuous(breaks = seq(-0.10, 0.10, 0.10)) +
        facet_wrap(~maf) +
        theme_classic() + 
        xlab('PC1') + ylab('PC2') +
        theme(axis.text = element_text(size = 13),
            axis.title = element_text(size = 15),
            legend.position = 'top')
pca_byContinent

In [41]:
# Save plot
outpath <- snakemake@output[[7]]
print(outpath)
ggsave(filename = outpath, plot = pca_byContinent, device = 'pdf', 
       dpi = 300, width = 14, height = 14, units = 'in')

#### Colored by range

In [42]:
# Plot
col1 <- wes_palette("Darjeeling1", n = 5, type = 'discrete')[2]
col2 <- wes_palette("Darjeeling1", n = 5, type = 'discrete')[4]
cols <- c(col1, col2)
pca_byRange <- eigen_df %>% 
    filter(sample_set == 'highErrorRemoved') %>% 
    ggplot(., aes(x = PC1, y = PC2, color = range, shape = site)) +
        geom_point(size = 2) + 
        scale_color_manual(values = cols) + 
        scale_x_continuous(breaks = seq(-0.10, 0.10, 0.10)) +
        facet_wrap(~maf) +
        theme_classic() + 
        xlab('PC1') + ylab('PC2') +
        theme(axis.text = element_text(size = 13),
            axis.title = element_text(size = 15),
            legend.position = 'top')
pca_byRange

In [43]:
# Save plot
outpath <- snakemake@output[[8]]
print(outpath)
ggsave(filename = outpath, plot = pca_byRange, device = 'pdf', 
       dpi = 300, width = 14, height = 14, units = 'in')

#### My take

- PCAs are the same, regardless of sample set
    - Suggests we could use all samples for this analysis (N = 515).
- Regardless of MAF, PCAs are basically the same
    - MAF = 0.05 is standard so I suggest we stick with this. 
- PC2 seems to largely separate Introduced vs. Native range
- Unclear what PC1 corresponds to (maybe related to something like Lat/Long?)
- Urban/rural sites seem to be largely overlapping within cities with some exceptions (e.g., Thessaloniki). 


## RDA

- Explicit test for urban/rural divergence in allele frequencies, conditioned on city
- Focus only on *highErrorRemoved* sample set and MAF = 0.05

In [44]:
highErrorRemoved_0.05_forRDA <- covMat_df %>% 
    filter(sample_set == 'highErrorRemoved' & maf == '0.05') %>% 
    bind_cols(., get_sample_sheet('highErrorRemoved'))

In [45]:
response_matrix <- highErrorRemoved_0.05_forRDA %>% 
    dplyr::select(starts_with('X')) %>% 
    as.matrix()
habitat_vector <- highErrorRemoved_0.05_forRDA %>% pull(site)
city_vector <- highErrorRemoved_0.05_forRDA %>% pull(city)

rda <- vegan::rda(response_matrix ~ habitat_vector + Condition(city_vector))

In [46]:
rda_summary <- summary(rda)

In [47]:
# Summary of RDA1 (i.e., habitat) and first 4 PCs
rda_summary$cont %>% as.data.frame() %>% dplyr::select(importance.RDA1:importance.PC4)

In [48]:
anova.cca(rda, by = 'terms', permutations = 1000, model = 'reduced')

#### My take:

- Permutation test suggests habitat explains significant variation in allele frequencies
    - However, variance explained byt habitat is very low (0.3%). 
- City explains 24% of the variation in allele frequencies (not shown since output is massive)